## **Carga y Tranformación de los datos**

Este notebook muestra el proceso de carga y transformación de los datos brindados por Stay Unique para la prueba técnica.

**Librerias necesarias:**

In [1]:
#importamos las librerias necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

**Carga de Archivos**
- Se brindaron 3 archivos en excel: Calendar, Properties y Bookings. 
- Los archivos se encuentran en formato Excel y divididos por pestañas

In [4]:
#Calendario
calendar_df = pd.read_excel("Datasets\data.xlsx", sheet_name="Calendar")
#Propiedades
properties_df = pd.read_excel("Datasets\data.xlsx", sheet_name="Properties")
#Reservas
booking_df =  pd.read_excel("Datasets\data.xlsx", sheet_name="Bookings")

### **Transformaciones de Calendar**

In [5]:
#Información de df Calendar:
calendar_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38893 entries, 0 to 38892
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   PropertyId          38893 non-null  int64         
 1   CalendarDate        38893 non-null  datetime64[ns]
 2   Blocked             38893 non-null  int64         
 3   Property_BookingId  31168 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 1.2 MB


- El dataframe contiene 4 columnas. 
Los tipos de datos de las columnas estan asignados correctamente, incluyendo el formato fecha de la columna CalendarDate. 
- Existen 38893 registros. 
- La columna Property_BookingId tiene datos nulos, se explorarán para determinar como trabajarlos.

In [6]:
calendar_df.sample(10)

,PropertyId,CalendarDate,Blocked,Property_BookingId
14838,43036,2022-07-22,2,103891.0
16683,43050,2022-07-27,2,103852.0
28580,43223,2022-11-29,2,110104.0
3059,4138,2022-07-19,2,103862.0
24688,43166,2022-11-01,2,109048.0
24588,43166,2022-07-24,0,NaN
25302,43170,2022-07-02,2,102751.0
38190,43366,2022-11-30,1,NaN
10441,42935,2022-08-10,2,104251.0
25269,43169,2022-11-30,2,109847.0


In [7]:
# Contar valores nulos de la columna 
print("\nValores nulos de la tabla Calendar")
print(calendar_df.isnull().sum())


Valores nulos de la tabla Calendar
PropertyId               0
CalendarDate             0
Blocked                  0
Property_BookingId    7725
dtype: int64


In [8]:
# Mascara para tener unicamente los valores nulos
null_booking = calendar_df[calendar_df['Property_BookingId'].isnull()]
# Con el filtro hecho determinamos a que valores de la columna Blocked corresponden los nulos
null_booking['Blocked'].value_counts()


0    6766
1     959
Name: Blocked, dtype: int64

Se determina que los valores nulos en la columna Property_BookingId unicamente corresponden a los valores 0 y 1 de la columna Blocked. Que representan aquellas propiedades que estan No reservadas y Bloqueadas respectivamente. 

Por lo que se puede realizar la transformación de estos valores nulos a 0, debido a que representan propiedades que no tienen Id de reserva.

In [9]:
#Convertimos los valores nulos de la columna Property_BookingID a 0, que determinan que no tienen reserva
calendar_df['Property_BookingId'].fillna(value= 0,inplace=True)

In [10]:
calendar_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38893 entries, 0 to 38892
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   PropertyId          38893 non-null  int64         
 1   CalendarDate        38893 non-null  datetime64[ns]
 2   Blocked             38893 non-null  int64         
 3   Property_BookingId  38893 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 1.2 MB


In [11]:
calendar_df.sample(5)

,PropertyId,CalendarDate,Blocked,Property_BookingId
26941,43194,2022-12-16,0,0.0
23741,43158,2022-10-05,2,107921.0
15906,43043,2022-12-17,2,111083.0
34425,43313,2022-11-04,2,108846.0
5333,4952,2022-09-23,2,108430.0


No es necesaria ninguna otra transformación

### **Transformaciones de Tabla Properties**

In [12]:
# Leemos el la tabla
properties_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 323 entries, 0 to 322
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   PropertyId  323 non-null    int64
 1   PR_Sleeps   323 non-null    int64
dtypes: int64(2)
memory usage: 5.2 KB


In [13]:
properties_df.sample(5)

,PropertyId,PR_Sleeps
186,43186,4
124,43075,5
20,43472,4
197,43214,6
73,42954,3


- Son 2 columnas, con 323 registros.
- No existen valores nulos y el formato de los campos es adecuado. 
- Se revisarán duplicados.

In [14]:
# Verificar duplicados en la tabla Propiedades
duplicates_in_propiedades = properties_df[properties_df.duplicated(subset='PropertyId', keep=False)]

# Imprimir las filas duplicadas (si existen)
if not duplicates_in_propiedades.empty:
    print("Filas duplicadas en la tabla Propiedades:")
    print(duplicates_in_propiedades)
else:
    print("No hay filas duplicadas en la tabla Propiedades.")

No hay filas duplicadas en la tabla Propiedades.


- No es necesaria ninguna transformación en la tabla

### **Transformaciones de tabla Bookings(Reservas)**

In [15]:
#Info de tabla Booking
booking_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6160 entries, 0 to 6159
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Property_BookingId         6160 non-null   int64         
 1   PB_TotalPrice              6160 non-null   float64       
 2   PB_NumNights               6160 non-null   int64         
 3   PB_BookingCreatedDate      6160 non-null   datetime64[ns]
 4   PB_BookingChannelOriginId  6160 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 240.8 KB


In [16]:
#Visualizamos la tabla Bookings
booking_df.sample(5)

,Property_BookingId,PB_TotalPrice,PB_NumNights,PB_BookingCreatedDate,PB_BookingChannelOriginId
2228,109872,523.73,4,2022-10-27 14:10:44,7
3814,111221,309.89,3,2022-12-04 19:24:50,7
5974,110357,404.40,3,2022-11-09 17:45:03,7
4073,108398,992.72,3,2022-09-22 18:13:12,7
2831,109005,255.60,2,2022-10-08 20:13:21,1


- Son 5 columnas con 6160 registros.
- Los formatos son correctos y no existen datos nulos en ninguna columna. 
- Se verificará duplicados en la columna Property_BookingId.

In [17]:
# Verificar duplicados en la tabla Bookings
duplicates_in_bookings = booking_df[booking_df.duplicated(subset='Property_BookingId', keep=False)]

# Imprimir las filas duplicadas (si existen)
if not duplicates_in_bookings.empty:
    print("Filas duplicadas en la tabla Bookings:")
    print(duplicates_in_bookings)
else:
    print("No hay filas duplicadas en la tabla Bookings.")

No hay filas duplicadas en la tabla Bookings.


No es necesario realizar limpieza o transformaciones.

### **Cargamos los datos**

In [ ]:
# Crear un escritor Excel (ExcelWriter)
clean_data = pd.ExcelWriter('clean_data.xlsx', engine='xlsxwriter')

# Guardar cada DataFrame en una pestaña diferente
calendar_df.to_excel(clean_data, sheet_name='Calendar', index=False)
properties_df.to_excel(clean_data, sheet_name='Properties', index=False)
booking_df.to_excel(clean_data, sheet_name='Bookings', index=False)

# Cerrar el escritor para finalizar el proceso de escritura
clean_data.save()